# Model Traning

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Modelling


In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [3]:
df=pd.read_csv('StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Preaparing X  and Y  varoables

In [4]:
x=df.drop(columns=['math score'],axis=1)
y=df['math score']

In [5]:
print("categories in 'gender' variable: ",end=" ")
print(df['gender'].unique())

print("categories in 'race/ethnicity' variable: ",end=" ")
print(df['race/ethnicity'].unique())

print("categories in 'parental level of education' variable: ",end=" ")
print(df['parental level of education'].unique())

print("categories in 'lunch' variable: ",end=" ")
print(df['lunch'].unique())

print("categories in 'test preparation course' variable: ",end=" ")
print(df['test preparation course'].unique())


categories in 'gender' variable:  ['female' 'male']
categories in 'race/ethnicity' variable:  ['group B' 'group C' 'group A' 'group D' 'group E']
categories in 'parental level of education' variable:  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
categories in 'lunch' variable:  ['standard' 'free/reduced']
categories in 'test preparation course' variable:  ['none' 'completed']


In [6]:
num_feature=x.select_dtypes(exclude='object').columns
cat_feature=x.select_dtypes(include='object').columns

In [7]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numerical_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprecessor=ColumnTransformer(

[
    ("OneHotEncoder",oh_transformer,cat_feature),
    ("numerical_transformer",numerical_transformer,num_feature)
]
)


In [8]:
x=preprecessor.fit_transform(x)

In [9]:
x


array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
def evaluate_model(true , predicted):
    mae=mean_absolute_error(true , predicted)
    mse=mean_squared_error(true , predicted)
    rmse=np.sqrt(mean_squared_error(true , predicted))
    r2_square=r2_score(true , predicted)
    return mae , mse , rmse , r2_square

In [12]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge ": Ridge(),
    "Lasso ": Lasso(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "Support Vector Regressor (SVR)": SVR(),
    "CatBoost Regressor": CatBoostRegressor(verbose=0),
    "XGBoost Regressor": XGBRegressor()
}
model_list=[]
r2_list=[]


In [13]:
for name, model in models.items():
    model.fit(x_train,y_train)
    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
    
    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train, train_pred)
    test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test, test_pred)

    r2_list.append((name, test_r2))
    r2_list.sort(key=lambda x: x[1], reverse=True)
    print(f"Model: {name}")
    print(f"  Training Data - MAE: {train_mae}, MSE: {train_mse}, RMSE: {train_rmse}, R^2: {train_r2}")
    
    print(f"  Testing Data  - MAE: {test_mae}, MSE: {test_mse}, RMSE: {test_rmse}, R^2: {test_r2}")
    print("==" * 40)
    print("")


Model: Linear Regression
  Training Data - MAE: 4.271484375, MSE: 28.4013037109375, RMSE: 5.329287354885032, R^2: 0.874022530822051
  Testing Data  - MAE: 4.2221875, MSE: 29.43263671875, RMSE: 5.425185408698029, R^2: 0.8790464771743793

Model: Ridge 
  Training Data - MAE: 4.264987823725981, MSE: 28.33778823308244, RMSE: 5.323324922741654, R^2: 0.8743042615212909
  Testing Data  - MAE: 4.21110068801426, MSE: 29.056272192348278, RMSE: 5.390387016935637, R^2: 0.8805931485028738

Model: Lasso 
  Training Data - MAE: 5.206302661246526, MSE: 43.47840400585577, RMSE: 6.593815587795565, R^2: 0.8071462015863458
  Testing Data  - MAE: 5.157881810347763, MSE: 42.5064168384116, RMSE: 6.519694535667419, R^2: 0.8253197323627853

Model: K-Neighbors Regressor
  Training Data - MAE: 4.51675, MSE: 32.57995, RMSE: 5.707884897227694, R^2: 0.8554876322327585
  Testing Data  - MAE: 5.621, MSE: 52.6066, RMSE: 7.253040741647602, R^2: 0.7838129945787431

Model: Decision Tree Regressor
  Training Data - MAE: 0

In [14]:
for model_name, r2_score in r2_list:
    print(f"Model: {model_name}, R^2: {r2_score}")

Model: Ridge , R^2: 0.8805931485028738
Model: Linear Regression, R^2: 0.8790464771743793
Model: Random Forest Regressor, R^2: 0.8538388949241513
Model: CatBoost Regressor, R^2: 0.8516318920747058
Model: AdaBoost Regressor, R^2: 0.843023122644107
Model: XGBoost Regressor, R^2: 0.8277965653186071
Model: Lasso , R^2: 0.8253197323627853
Model: K-Neighbors Regressor, R^2: 0.7838129945787431
Model: Decision Tree Regressor, R^2: 0.751005184549582
Model: Support Vector Regressor (SVR), R^2: 0.7286001513223704


In [19]:
from sklearn.metrics import r2_score
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(x_train, y_train)
y_pred = lin_model.predict(x_test)
score = r2_score(y_test, y_pred)
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 0.88
